In [27]:
import cv2
import numpy as np


In [28]:
# Load the image
filename = "resistor3.png"
image = cv2.imread(filename)

cv2.imshow("Image", image)
cv2.waitKey(0) 

# edge detection
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 20, 150)

cv2.imshow("Edges", edges)
cv2.waitKey(0)

# find center line of the resistor
lines = cv2.HoughLines(edges, 1, np.pi / 180, 50)

# Draw the lines
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho

    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))

    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

cv2.imshow("Image", image)
cv2.waitKey(0)

# try to detect 4 color bands on the center line
# get the center line
center_line = (x1, y1), (x2, y2)

# crop image to the center line
image = cv2.imread(filename)
image = image[center_line[0][1]:center_line[1][1], center_line[0][0]:center_line[1][0]]

# color detection on the center line
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

cv2.imshow("Image", image)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [29]:
import numpy as np

# resize the image to 32 x 1
image = cv2.resize(image, (32, 1))
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)


# Display the image
cv2.imshow("Image", image)
cv2.waitKey(0)

# find contiguous regions of the similar color within +-5 hue color; for each region, find the average color
def find_similar_colors(hsv_image):
    colors = []
    for i in range(0, 32):
        if i == 0:
            continue
        if i == 31:
            continue
        if abs(hsv_image[0][i][0] - hsv_image[0][i-1][0]) < 5:
            if len(colors) == 0:
                colors.append([hsv_image[0][i][0], hsv_image[0][i][1], hsv_image[0][i][2]])
            else:
                colors[-1][0] = (colors[-1][0] + hsv_image[0][i][0]) / 2
                colors[-1][1] = (colors[-1][1] + hsv_image[0][i][1]) / 2
                colors[-1][2] = (colors[-1][2] + hsv_image[0][i][2]) / 2
        else:
            colors.append([hsv_image[0][i][0], hsv_image[0][i][1], hsv_image[0][i][2]])


    return colors

colors = find_similar_colors(hsv_image)
print("Colors: ", colors)

cv2.imshow("Image", image)
cv2.waitKey(0)


cv2.destroyAllWindows()

C:\Users\user\AppData\Local\Temp\ipykernel_9044\2120407992.py:26: RuntimeWarning: overflow encountered in scalar add
  colors[-1][2] = (colors[-1][2] + hsv_image[0][i][2]) / 2
C:\Users\user\AppData\Local\Temp\ipykernel_9044\2120407992.py:20: RuntimeWarning: overflow encountered in scalar subtract
  if abs(hsv_image[0][i][0] - hsv_image[0][i-1][0]) < 5:


Colors:  [[28.0, 25.5, 43.0], [26, 25, 171], [21, 58, 184], [20.0, 70.0, 101.5], [18, 93, 220], [10, 159, 119], [5, 135, 102], [19, 101, 225], [25, 107, 213], [60, 196, 207], [54, 203, 195], [20, 104, 219], [11, 134, 251], [0, 255, 253], [9, 135, 248], [19.875, 112.53125, 215.9375], [20.625, 70.625, 183.5], [29.5, 30.5, 43.5], [28, 26, 170]]
